In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

d:\App\anaconda\envs\toolkit\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#movies data
movies = tfds.load("movielens/100k-movies", split="train")
#ratings data
ratings = tfds.load("movielens/100k-ratings", split="train")

In [3]:
def sort_ratings(ds):
    timestamps = []
    features_list = []

    for example in ds:
        timestamps.append(example['timestamp'])
        features_list.append(example)

    # convert list to tensor
    timestamps = tf.convert_to_tensor(timestamps)
    features_list = tf.stack(features_list)
    # sort index
    sorted_indices = tf.argsort(timestamps)
    # sort all data
    sorted_features = tf.gather(ds, sorted_indices)
    return tf.data.Dataset.from_tensor_slices(sorted_features)

In [4]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

In [5]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [6]:
movie_titles = movies.batch(2_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

In [7]:
unique_movie_titles = np.unique(list(movie_titles))
unique_user_ids = np.unique(list(user_ids))

unique_user_ids[:10]

array([b'1', b'10', b'100', b'101', b'102', b'103', b'104', b'105',
       b'106', b'107'], dtype=object)

In [8]:
embedding_dimension = 32

user_model = tf.keras.Sequential([
    tf.keras.layers.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
    tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

In [9]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.StringLookup(
      vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

In [10]:
metrics = tfrs.metrics.FactorizedTopK(
  candidates=movies.batch(128).map(movie_model)
)

In [11]:
task = tfrs.tasks.Retrieval(
  metrics=metrics
)

In [12]:
from typing import Dict, Text

class MovielensModel(tfrs.Model):

  def __init__(self, user_model, movie_model):
    super().__init__()
    self.movie_model: tf.keras.Model = movie_model
    self.user_model: tf.keras.Model = user_model
    self.task: tf.keras.layers.Layer = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False):
    # pick out the user features and pass them into the user model.
    user_embeddings = self.user_model(features["user_id"])
    # and pick out the movie features and pass them into the movie model,
    # getting embeddings back.
    positive_movie_embeddings = self.movie_model(features["movie_title"])

    # computes the loss and the metrics.
    return self.task(user_embeddings, positive_movie_embeddings)

In [13]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [14]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [15]:
model.fit(cached_train, epochs=5)

Epoch 1/5
10/10 [==============================] - 6s 412ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0012 - factorized_top_k/top_5_categorical_accuracy: 0.0106 - factorized_top_k/top_10_categorical_accuracy: 0.0225 - factorized_top_k/top_50_categorical_accuracy: 0.1061 - factorized_top_k/top_100_categorical_accuracy: 0.1852 - loss: 69816.4716 - regularization_loss: 0.0000e+00 - total_loss: 69816.4716
Epoch 2/5
10/10 [==============================] - 4s 383ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0031 - factorized_top_k/top_5_categorical_accuracy: 0.0199 - factorized_top_k/top_10_categorical_accuracy: 0.0411 - factorized_top_k/top_50_categorical_accuracy: 0.1736 - factorized_top_k/top_100_categorical_accuracy: 0.2994 - loss: 67406.7017 - regularization_loss: 0.0000e+00 - total_loss: 67406.7017
Epoch 3/5
10/10 [==============================] - 4s 382ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0037 - factorized_top_k/top_5_categorical_accuracy: 

In [16]:
model.evaluate(cached_test, return_dict=True)

5/5 [==============================] - 1s 148ms/step - factorized_top_k/top_1_categorical_accuracy: 7.5000e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0064 - factorized_top_k/top_10_categorical_accuracy: 0.0146 - factorized_top_k/top_50_categorical_accuracy: 0.1066 - factorized_top_k/top_100_categorical_accuracy: 0.2149 - loss: 31177.3467 - regularization_loss: 0.0000e+00 - total_loss: 31177.3467


{'factorized_top_k/top_1_categorical_accuracy': 0.000750000006519258,
 'factorized_top_k/top_5_categorical_accuracy': 0.006399999838322401,
 'factorized_top_k/top_10_categorical_accuracy': 0.014600000344216824,
 'factorized_top_k/top_50_categorical_accuracy': 0.10655000060796738,
 'factorized_top_k/top_100_categorical_accuracy': 0.21490000188350677,
 'loss': 28311.638671875,
 'regularization_loss': 0,
 'total_loss': 28311.638671875}

In [17]:
model.user_model.weights

[<tf.Variable 'embedding/embeddings:0' shape=(944, 32) dtype=float32, numpy=
 array([[ 0.0382635 , -0.03688531, -0.03190091, ..., -0.04987667,
          0.04244686, -0.04079776],
        [ 0.01157301, -0.194407  ,  0.00535114, ..., -0.1925165 ,
         -0.21170756, -0.55986273],
        [-0.00120608, -0.04744984,  0.17872335, ...,  0.27430916,
         -0.17080961, -0.01066757],
        ...,
        [ 0.43189016, -0.21493812, -0.24408276, ..., -0.21796802,
         -0.06237882,  0.4411375 ],
        [ 0.73038626, -0.47296962,  0.14744417, ..., -0.52264464,
          0.02744576,  0.5616209 ],
        [-0.18957417,  0.50170344, -0.01452157, ...,  0.23997432,
         -0.06825229, -0.19644788]], dtype=float32)>]

In [21]:
# create a model that takes in raw query features, and
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
# recommends movies out of the entire movies dataset.
index.index_from_dataset(
  movies.batch(128).map(lambda title: (title, model.movie_model(title)))
)

# get recommendations.
_, titles = index(np.array(["21"]), k=3)

<tf.Tensor: shape=(), dtype=string, numpy=b'Believers, The (1987)'>